### Gather data for PCA

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio 
from pathlib import Path
from scipy import ndimage
from scipy.misc import bytescale


In [2]:
#inputs local
inortho = r"T:\UAS\2018-676-FA\products\ortho\2018-10-23-Wildlands_ortho.tif"
indsm = r"T:\UAS\2018-676-FA\products\dsm\2018-10-23-Wildlands_DSM_10cm.tif"
invari = r"D:\jlogan\2018-676-FA\tmp_veg_indices\vari_scaled_int_fromlzw.tif"
intgi = r"D:\jlogan\2018-676-FA\tmp_veg_indices\tgi_scaled_int_fromlzw.tif"
inpoints = r"T:\UAS\2018-676-FA\validation\wld_topo_17-18_combined_kdthinned10cm.csv"

#inputs lab computer
#inortho = r"D:\jlogan\2018-676-FA\products\ortho\2018-10-23-Wildlands_ortho_lzw.tif"
#indsm = r"D:\jlogan\2018-676-FA\products\dsm\2018-10-23-Wildlands_DSM_10cm.tif"
#invari = r"D:\jlogan\2018-676-FA\tmp_veg_indices\vari_scaled_int_fromlzw.tif"
#intgi = r"D:\jlogan\2018-676-FA\tmp_veg_indices\tgi_scaled_int_fromlzw.tif"
#inpoints = r"T:\UAS\2018-676-FA\validation\wld_topo_17-18_combined_kdthinned10cm.csv"

for n in [inortho, indsm, invari, intgi, inpoints]:
    n = Path(n)   

In [3]:
#load input pts
ptdf = pd.read_csv(inpoints)

In [4]:
#load input rgb image
dataset = rasterio.open(inortho)
aff = dataset.transform
#img = dataset.read()

In [5]:
r = dataset.read(1)
g = dataset.read(2)
b = dataset.read(3)

In [ ]:
#calc psuedo veg indices
#https://agribotix.com/blog/2017/04/30/comparing-rgb-based-vegetation-indices-with-ndvi-for-agricultural-drone-imagery/
tgi = g - 0.39 * r - 0.61 * b
vari = (g-r)/g+r-b

#mask infs in vari
vari_masked = np.ma.masked_invalid(vari)
tgi_masked = np.ma.masked_invalid(tgi)
del tgi, vari

#scale to 0 -255
vari_scaled = bytescale(vari_masked)
tgi_scaled = bytescale(tgi_masked)
del tgi_masked, vari_masked


C:\Users\jlogan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\jlogan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\jlogan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `bytescale` is deprecated!
`bytescale` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
  if sys.path[0] == '':
C:\Users\jlogan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `bytescale` is deprecated!
`bytescale` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
  del sys.path[0]


In [ ]:
r.shape, vari_scaled.shape

In [43]:
#for r, g, b, vari and tgi sample grid and add to df

#get img coords on df
ptdf['imgcol_int'], ptdf['imgrow_int'] = np.floor(~aff * (ptdf['e'], ptdf['n'])).astype(int)

ptdf['r'] = r[ptdf['imgrow_int'],ptdf['imgcol_int']]
ptdf['g'] = g[ptdf['imgrow_int'],ptdf['imgcol_int']]
ptdf['b'] = b[ptdf['imgrow_int'],ptdf['imgcol_int']]
ptdf['vari'] = vari_scaled[ptdf['imgrow_int'],ptdf['imgcol_int']]
ptdf['tig'] = tgi_scaled[ptdf['imgrow_int'],ptdf['imgcol_int']]

In [48]:
#load dem


#calc mean of kernel
#from https://gis.stackexchange.com/a/254795/129277
m = np.array([[13, 21, 13,  8],
              [ 5, 10, 22, 14],
              [21, 33,  9,  0],
              [ 0,  0,  0,  0]], dtype=np.float)

result = ndimage.generic_filter(a, np.nanmean, size=3, mode='constant', cval=np.NaN)
